In [1]:
#Import all reuired libraries
import pandas as pd
import datetime
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
from datetime import timedelta
from sklearn import preprocessing
import category_encoders as ce 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import os


PROJ: proj_create_from_database: Cannot find proj.db


The primary dataset which will be used in this analysis is the victim based part 1 crime table, produced by the Baltimore City Police Department. It reports crimes based upon each victim, rather than occurance, and thus each row can be thought of as a victim. 

In [2]:
Crime = pd.read_csv('https://data.baltimorecity.gov/api/views/wsfq-mvij/rows.csv?accessType=DOWNLOAD')
#Crime = pd.read_csv('C:/Users/cyeager/Desktop/UMBC/DATA 602/Homework2/Crime.csv',sep=",")

In [4]:
print(Crime.columns)

Index(['CrimeDate', 'CrimeTime', 'CrimeCode', 'Location', 'Description',
       'Inside/Outside', 'Weapon', 'Post', 'District', 'Neighborhood',
       'Longitude', 'Latitude', 'Location 1', 'Premise', 'vri_name1',
       'Total Incidents'],
      dtype='object')


In [5]:
#Keep only the columns we want
df=Crime.filter(['CrimeDate', 'CrimeTime','Description','Latitude','Longitude'])

In [6]:
#Concat the date and time columns into a single feature
df['DT']=df['CrimeDate'] + ' ' + df['CrimeTime']
#df

In [7]:
#reformat into a new datetime column
df['DT']=pd.to_datetime(df['DT'])
#df

Here we zip the latitude and longitude features into one, then apply the point function so that they can be displayed as a single dot of a map for each crime.

In [8]:
df['Coordinates'] = list(zip(df['Longitude'], df['Latitude']))
df['Coordinates'] = df['Coordinates'].apply(Point)

In [10]:
df.head(5)

,CrimeDate,CrimeTime,Description,Latitude,Longitude,DT,Coordinates
0,11/28/2020,10:08:00,SHOOTING,39.338127,-76.667398,2020-11-28 10:08:00,POINT (-76.667398141996 39.338126631537)
1,11/28/2020,05:51:00,LARCENY FROM AUTO,39.286621,-76.608625,2020-11-28 05:51:00,POINT (-76.608625261266 39.286621250631)
2,11/28/2020,23:40:00,LARCENY,39.310340,-76.637774,2020-11-28 23:40:00,POINT (-76.63777365973699 39.310339749999)
3,11/28/2020,07:00:00,LARCENY,39.316573,-76.647512,2020-11-28 07:00:00,POINT (-76.64751204505201 39.316573452184)
4,11/28/2020,23:17:00,AGG. ASSAULT,39.318264,-76.601968,2020-11-28 23:17:00,POINT (-76.60196811639301 39.318263972228)


In [177]:
#All data prior to Jan 1 2014 was found to be sparse and inconsistent and thus is removed
df2=df[(df['CrimeDate'] == '11/28/2020')]
print(df2['DT'].min())
print(df2['DT'].max())

2020-11-28 00:00:00
2020-11-28 23:45:00


In [178]:
#Save of a copy of this to come back to if we mess it up
df2['hour']=df2['DT'].dt.hour

C:\Users\cyeager\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [179]:
df2=df2.sort_values('DT')
df2

,CrimeDate,CrimeTime,Description,Latitude,Longitude,DT,Coordinates,hour
17,11/28/2020,00:00:00,LARCENY,39.252843,-76.648608,2020-11-28 00:00:00,POINT (-76.64860834329799 39.252843283579),0
20,11/28/2020,00:00:00,LARCENY,39.314251,-76.619431,2020-11-28 00:00:00,POINT (-76.61943073476499 39.314250935432),0
31,11/28/2020,02:30:00,COMMON ASSAULT,39.305811,-76.655792,2020-11-28 02:30:00,POINT (-76.65579216050399 39.305810728027),2
30,11/28/2020,04:30:00,AGG. ASSAULT,39.327472,-76.683064,2020-11-28 04:30:00,POINT (-76.683064276595 39.327471912233),4
5,11/28/2020,05:06:00,BURGLARY,39.285037,-76.579921,2020-11-28 05:06:00,POINT (-76.579921389313 39.285037036427),5
21,11/28/2020,05:51:00,LARCENY FROM AUTO,39.286621,-76.608625,2020-11-28 05:51:00,POINT (-76.608625261266 39.286621250631),5
1,11/28/2020,05:51:00,LARCENY FROM AUTO,39.286621,-76.608625,2020-11-28 05:51:00,POINT (-76.608625261266 39.286621250631),5
22,11/28/2020,05:55:00,LARCENY FROM AUTO,39.286666,-76.607531,2020-11-28 05:55:00,POINT (-76.607530552025 39.286665705829),5
24,11/28/2020,06:11:00,LARCENY,39.355572,-76.701401,2020-11-28 06:11:00,POINT (-76.701401200861 39.355572376101),6
11,11/28/2020,06:29:00,BURGLARY,39.287875,-76.651532,2020-11-28 06:29:00,POINT (-76.651531928506 39.287875416065),6


In [180]:
df2.Description.value_counts()

LARCENY                 13
COMMON ASSAULT           9
AGG. ASSAULT             8
AUTO THEFT               5
LARCENY FROM AUTO        3
BURGLARY                 2
SHOOTING                 2
ROBBERY - STREET         1
ROBBERY - COMMERCIAL     1
Name: Description, dtype: int64

In [181]:
Frame=pd.date_range("00:00", "23:59", freq="10min").strftime('%H:%M:%S')

In [182]:
len(Frame)-13

131

In [183]:
Frame[0:13]

Index(['00:00:00', '00:10:00', '00:20:00', '00:30:00', '00:40:00', '00:50:00',
       '01:00:00', '01:10:00', '01:20:00', '01:30:00', '01:40:00', '01:50:00',
       '02:00:00'],
      dtype='object')

In [184]:
Frame[0]

'00:00:00'

In [185]:
Frame[12]

'02:00:00'

In [186]:
df2[(df2.CrimeTime >=Frame[0]) & (df2.CrimeTime <=Frame[12])]

,CrimeDate,CrimeTime,Description,Latitude,Longitude,DT,Coordinates,hour
17,11/28/2020,00:00:00,LARCENY,39.252843,-76.648608,2020-11-28,POINT (-76.64860834329799 39.252843283579),0
20,11/28/2020,00:00:00,LARCENY,39.314251,-76.619431,2020-11-28,POINT (-76.61943073476499 39.314250935432),0


In [187]:
OneDay=[]
Ranges=[]
i=0
ii=12
for i in range(132):
    OneDay.append(df2[(df2.CrimeTime >=Frame[i]) & (df2.CrimeTime <=Frame[ii])])
    Ranges.append(Frame[i]+'to'+Frame[ii])
    i+=1
    ii+=1
    
    #print(i)
    #print(ii)

In [188]:
#We want each type of crime to be a specific color
ClusterColors = ['#E47A2E', '#3F69AA', '#D5AE41', '#BD3D3A','#BD3D3A',"#006E6D",'#7F4145','#223A5E','#BD3D3A', '#3F69AA', '#D5AE41', '#766F57','#E47A2E',"#006E6D",'#7F4145','#223A5E',"#9C9A40","#9C9A40",'blue','green','red','purple','brown','#BD3D3A', '#3F69AA', '#D5AE41', '#766F57','#E47A2E',"#006E6D",'#7F4145','#223A5E',"#9C9A40"]

In [189]:
#Here we assign the color and size to each crime type point
CrimeColours = list(zip(list(df2['Description'].unique()), ClusterColors))
CrimeColours=pd.DataFrame(CrimeColours)
CrimeColours['shape']=50

In [190]:
#Rename Colmns
CrimeColours.columns=['crime','color','shape']

In [191]:
df2 = pd.merge(df2, CrimeColours, left_on=["Description"], right_on=['crime'])
df2

,CrimeDate,CrimeTime,Description,Latitude,Longitude,DT,Coordinates,hour,crime,color,shape
0,11/28/2020,00:00:00,LARCENY,39.252843,-76.648608,2020-11-28 00:00:00,POINT (-76.64860834329799 39.252843283579),0,LARCENY,#E47A2E,50
1,11/28/2020,00:00:00,LARCENY,39.314251,-76.619431,2020-11-28 00:00:00,POINT (-76.61943073476499 39.314250935432),0,LARCENY,#E47A2E,50
2,11/28/2020,06:11:00,LARCENY,39.355572,-76.701401,2020-11-28 06:11:00,POINT (-76.701401200861 39.355572376101),6,LARCENY,#E47A2E,50
3,11/28/2020,07:00:00,LARCENY,39.316573,-76.647512,2020-11-28 07:00:00,POINT (-76.64751204505201 39.316573452184),7,LARCENY,#E47A2E,50
4,11/28/2020,07:00:00,LARCENY,39.295948,-76.575974,2020-11-28 07:00:00,POINT (-76.57597377324099 39.295947884832),7,LARCENY,#E47A2E,50
5,11/28/2020,10:16:00,LARCENY,39.337493,-76.634283,2020-11-28 10:16:00,POINT (-76.63428263311201 39.337493180823),10,LARCENY,#E47A2E,50
6,11/28/2020,10:20:00,LARCENY,39.293293,-76.663424,2020-11-28 10:20:00,POINT (-76.66342396812399 39.293292940633),10,LARCENY,#E47A2E,50
7,11/28/2020,16:58:00,LARCENY,39.328243,-76.608376,2020-11-28 16:58:00,POINT (-76.608376111609 39.328243111609),16,LARCENY,#E47A2E,50
8,11/28/2020,17:30:00,LARCENY,39.355503,-76.701537,2020-11-28 17:30:00,POINT (-76.701536692603 39.355502661618),17,LARCENY,#E47A2E,50
9,11/28/2020,18:30:00,LARCENY,39.318631,-76.653986,2020-11-28 18:30:00,POINT (-76.65398557534199 39.318631252038),18,LARCENY,#E47A2E,50


In [192]:
OneDay=[]
Ranges=[]
i=0
ii=12
for i in range(132):
    OneDay.append(df2[(df2.CrimeTime >=Frame[i]) & (df2.CrimeTime <=Frame[ii])])
    Ranges.append(Frame[i]+'to'+Frame[ii])
    i+=1
    ii+=1
    
    #print(i)
    #print(ii)

In [193]:
searchfor=["SHOOTING","HOMICIDE"]
um=[]
for i in range(len(OneDay)):
    
    um.append(OneDay[i][OneDay[i]['Description'].str.contains('|'.join(searchfor))].empty)


In [194]:
#Read in the neighborhoods boundary shapefile from API
BaltimoreCommunities = gpd.read_file('https://data.baltimorecity.gov/api/geospatial/peh2-3qyi?method=export&format=Shapefile')

In [195]:
dud=[]
for i in range(len(OneDay)):
    dud.append(OneDay[i].empty)

In [196]:
skiplist=[i for i,x in enumerate(dud) if x == True]
#skiplist

In [197]:
#Only 137 of the nearly 30k 2 hour windows contained no crime.
len(skiplist)

3

In [198]:
print(os.getcwd()+"\Maps\TestRun"+"\\"+str(um[i])+"\\"+Frame[i].replace(":", "")+"_"+str(um[i])+".png")

C:\Users\cyeager\Desktop\UMBC\DATA 602\Final\Maps\TestRun\True\215000_True.png


In [199]:
for i in range(0,len(OneDay)):
    if OneDay[i].empty==True:
        continue    
    base = BaltimoreCommunities.plot(color='none', edgecolor='black',alpha=.5)

    base.set_aspect('equal')
    base.set_axis_off()

    ax=gpd.GeoDataFrame(OneDay[i], geometry='Coordinates').set_crs("EPSG:4326").plot(ax=base,c=OneDay[i]['color'], markersize = OneDay[i]['shape'],alpha=.7)

    gpd.sjoin(BaltimoreCommunities,gpd.GeoDataFrame(OneDay[i], geometry='Coordinates').set_crs("EPSG:4326"), op='contains').plot(ax=base,edgecolor='red',color='none')
  
    fig = ax.figure


    File=(os.getcwd()+"\Maps\TestRun"+"\\"+str(um[i])+"\\"+Frame[i].replace(":", "")+"_"+str(um[i])+".png")
    fig.savefig(File, bbox_inches = 'tight',pad_inches = 0)
    plt.close('all')